In [1]:
import itertools
import numpy as np
import scipy.special as ss

In [2]:
p_vals = np.arange(1,20)
n_vals = np.arange(1,20)
e_vals = np.arange(33)

attack_length = 2
e_lists = list(itertools.product(*[e_vals]*attack_length))

# cartesian product of possible values of p, n, (e_{ell-1}, e_ell)
vectors = list(itertools.product(*[p_vals, n_vals, e_lists]))

In [3]:
def calcDelayDifference(p, n, e_list):
    e_ell = e_list[1]
    return 40 * (p - n) + 8 * max(24-e_ell, 0) - 8 * max(e_ell-8, 0)

def calcRewardDifference(p, e_list):
    block_reward = 16 * (1/(p+1) + 0.8 + 0.2 * (e_list[1] / 32) - 1)
    endorse_reward = 2 * e_list[0] * (1/(p+1) - 1)
    return block_reward + endorse_reward

def calcProb(alpha, p, n, e_list):
    binoms = [ss.binom(32, e) for e in e_list]
    return np.product(binoms) * np.power(alpha, n + sum(e_list) + 1) * np.power(1- alpha,65  + p - sum(e_list))

In [4]:
# minutes in a year
C = 365 * 24 * 60
for alpha in np.arange(0.1, 0.45, 0.05):
    probabilites = []
    values = []
    for p, n, e_list in vectors:
        reward = calcRewardDifference(p, e_list)
        # if feasible and profitable
        if (calcDelayDifference(p, n, e_list) < 0) and (reward > 0):
            prob = calcProb(alpha, p, n, e_list)
            probabilites.append(prob)
            values.append(prob * reward)
    # expected reward per minute is alpha * (block reward + total endorse reward)
    expected_reward = alpha * ( 16 +  2 * 32 )
    percent_increase = 100 * sum(values) / expected_reward 
    
    print(alpha, sum(probabilites) * C, sum(values) * C, percent_increase)

0.1 0.1400478103939502 0.3481601176903832 8.280063681753786e-06
0.15000000000000002 3.1441967897518857 5.963641278394176 9.45529122018356e-05
0.20000000000000004 22.1276708060534 34.854770510535786 0.0004144640709491031
0.25000000000000006 77.32232288231205 100.17002304086098 0.000952911178090382
0.30000000000000004 152.09882475542113 160.2309862778126 0.0012702228110557188
0.3500000000000001 172.4561129736277 153.66326313120047 0.0010441350234507531
0.40000000000000013 115.17202728903314 92.6669383674577 0.0005509592511383281
